In [19]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import datetime
import collections
# spark = SparkSession\
#     .builder\
#     .appName("MergeDataset")\
#     .getOrCreate()

In [20]:
player_attr = pd.read_csv('./data/player_attr.csv',keep_default_na=False)
#Bị sai, đáng ra phải lấy  fbref_MatchSquad
fbref_MatchSquad = pd.read_csv('./data/fbref_MatchSquad.csv',keep_default_na=False)
fbref_MatchInfos = pd.read_csv('./data/fbref_MatchInfos.csv',keep_default_na=False)
# player_attr.fillna(0)

C:\Users\super\AppData\Local\Temp\ipykernel_2276\2567350895.py:1: DtypeWarning: Columns (10,13,29) have mixed types. Specify dtype option on import or set low_memory=False.
  player_attr = pd.read_csv('./data/player_attr.csv',keep_default_na=False)


In [21]:
player_attr[player_attr.columns.values.tolist()] = player_attr[player_attr.columns.values.tolist()].astype(str)
fbref_MatchSquad[fbref_MatchSquad.columns.values.tolist()] = fbref_MatchSquad[fbref_MatchSquad.columns.values.tolist()].astype(str)
fbref_MatchInfos[fbref_MatchInfos.columns.values.tolist()] = fbref_MatchInfos[fbref_MatchInfos.columns.values.tolist()].astype(str)
fbref_MatchInfos['match_date'] = fbref_MatchInfos['match_date'].astype('datetime64[ns]')
fbref_MatchSquad

,fbrefMatchId,is_home_team,is_sub,player_kitnum,player_name,team
0,db4d7668,Yes,No,26,Ben Foster,Watford
1,db4d7668,Yes,No,7,Gerard Deulofeu,Watford
2,db4d7668,Yes,No,9,Troy Deeney,Watford
3,db4d7668,Yes,No,15,Craig Cathcart,Watford
4,db4d7668,Yes,No,16,Abdoulaye Doucouré,Watford
...,...,...,...,...,...,...
76289,e62f6e78,Yes,Yes,17,Cédric Soares,Arsenal
76290,e62f6e78,Yes,Yes,19,Nicolas Pépé,Arsenal
76291,e62f6e78,Yes,Yes,23,Albert Sambi Lokonga,Arsenal
76292,e62f6e78,Yes,Yes,24,Reiss Nelson,Arsenal


In [22]:
list_names = ['Troy','Deeney']

len([all(name in player_name for name in list_names) for player_name in player_attr['player_full_name'].tolist()])
len(player_attr)

85427

In [23]:
# This function merges the data from fbref, SoFifa and player attributes to create a merged dataset.
# It takes three parameters: fbref_MatchInfos, fbref_MatchPlayerStats and player_attr.
# It creates a dataframe called MergedSoFifaFbref and merges the data from fbref, SoFifa and player attributes.
# It then exports the merged dataset to a csv file called MergedSoFifaFbref.csv.
def MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr):
    # Create a dataframe called fbref_players which contains the player's kit number, name, position, team, nationality, fbrefMatchId and age from fbref_MatchPlayerStats.
    fbref_players = fbref_MatchPlayerStats[['player_kitnum','player_name','team','fbrefMatchId','is_home_team','is_sub']]
    # Merge the dataframe fbref_players with fbref_MatchInfos on the fbrefMatchId.
    fbref_players = fbref_players.merge(fbref_MatchInfos[['match_date','fbrefMatchId']], on='fbrefMatchId', how = 'inner')
    # Convert the player's name, match_date, age and update_date to strings.
    fbref_players['player_name'] = fbref_players['player_name'].astype('str')
    fbref_players['match_date'] = fbref_players['match_date'].astype('datetime64[ns]')
    player_attr['update_date'] = player_attr['update_date'].astype('datetime64[ns]')
    # Create a list of the columns of the merged dataset.
    columns_fbref_players = fbref_players.columns.values.tolist()
    columns_sofifa = ['birthday','nationality','overall_rating','player_full_name','player_name','sofifa_id','all_positions','team1','team1_kitnum','team2','team2_kitnum','update_date',]
    columns_MergedSoFifaFbref = list(columns_fbref_players)
    columns_MergedSoFifaFbref.extend(columns_sofifa)
    # Remove any duplicate columns from the list.
    columns_MergedSoFifaFbref = np.unique(columns_MergedSoFifaFbref).tolist()
    # Create a dataframe called MergedSoFifaFbref with the columns from the list.
    MergedSoFifaFbref = pd.DataFrame(columns=columns_MergedSoFifaFbref)
    # Iterate through each row of fbref_players.
    for i in range(0,len(fbref_players)):
    # for i in range(0,50):
        # Create a dataframe called fbref_player which contains the current row of fbref_players.
        fbref_player = fbref_players.iloc[i]
        # Split the player's name into separate words.
        fbref_player_name = fbref_player['player_name']
        fbref_player_name = fbref_player_name.split(' ')
        if ((fbref_player_name == '') or (fbref_player_name == 'nan') or any(pd.isna(fbref_player_name)) or any(pd.isnull(fbref_player_name))):
            continue
        # Get variables from the current row of fbref_players.
        match_date = fbref_player['match_date']
        try:
            player_kitnum = str(int(float(fbref_player['player_kitnum'])))
        except:
            continue
            
        team = fbref_player['team']
        # Filter Rơws in player_attr that have player_attr.player_full_name == fbref_player_name
        # rows_contain_name = [all(name in player_full_name for name in fbref_player_name) for player_full_name in player_attr['player_full_name'].tolist()]
        rows_contain_name = [all(name in player_full_name for name in fbref_player_name) for player_full_name in player_attr['player_full_name'].tolist()]
        df_sofifa = player_attr[rows_contain_name][columns_sofifa]
        if (len(df_sofifa)==0):
            rows_contain_name = [any(name in player_full_name for name in fbref_player_name) for player_full_name in player_attr['player_full_name'].tolist()]
            df_sofifa = player_attr[rows_contain_name][columns_sofifa]
        if (len(df_sofifa)==0):
            rows_contain_name = [any(name in player_full_name for name in fbref_player_name) for player_full_name in player_attr['player_name'].tolist()]
            df_sofifa = player_attr[rows_contain_name][columns_sofifa]
        # df_sofifa = df_sofifa.fillna(0)
        # Filter the dataframe df_sofifa to only contain the current month and year of the match_date.
        df_sofifa_old = df_sofifa.copy()
        df_sofifa = df_sofifa[(df_sofifa['update_date'].dt.month == match_date.month) & (df_sofifa['update_date'].dt.year == match_date.year)]
        if (len(df_sofifa)==0):
            df_sofifa = df_sofifa_old[df_sofifa_old['update_date'].dt.year == match_date.year]
        if (len(df_sofifa) == 0):
            df_sofifa = df_sofifa_old.copy()
        # Filter the dataframe df_sofifa to only contain the player's kit number and team.
        df_sofifa['team1_kitnum'] = df_sofifa['team1_kitnum'].astype(str)
        df_sofifa['team2_kitnum'] = df_sofifa['team2_kitnum'].astype(str)
        df_sofifa['team1'] = df_sofifa['team1'].astype(str)
        df_sofifa['team2'] = df_sofifa['team2'].astype(str)
        df_sofifa_old = df_sofifa.copy()
        df_sofifa = df_sofifa[(df_sofifa['team1_kitnum'] == player_kitnum) | (df_sofifa['team2_kitnum'] == player_kitnum)]
        df_sofifa = df_sofifa[(df_sofifa['team1'] == team) | (df_sofifa['team2'] == team)]
        if (len(df_sofifa)==0):
            df_sofifa = df_sofifa_old.copy()
        # Try to convert the age variable to an integer.
        # try:
        #     age = int(age)
        #     df_sofifa = df_sofifa[df_sofifa['age'] == age]
        # # If the conversion fails, pass.
        # except:
        #     pass
        # If the dataframe df_sofifa is not empty, create a dataframe called MergedSoFifaFbref_row which contains the current row of df_sofifa and the current row of fbref_players.
        if (len(df_sofifa)>0):
            MergedSoFifaFbref_row = df_sofifa.iloc[0:1]
            # Create a list of the columns of fbref_players and df_sofifa.
            delete_columns = fbref_players.columns.values.tolist()
            delete_columns.extend(MergedSoFifaFbref_row.columns.values.tolist())
            # Remove any duplicate columns from the list.
            delete_columns = [item for item, count in collections.Counter(delete_columns).items() if count > 1]
            MergedSoFifaFbref_row = MergedSoFifaFbref_row.drop(columns = delete_columns)
            # Reset the index of MergedSoFifaFbref_row.
            MergedSoFifaFbref_row.reset_index(drop=True, inplace=True)
            # Concatenate the current row of MergedSoFifaFbref_row and the current row of fbref_players.
            # MergedSoFifaFbref_row = pd.concat([MergedSoFifaFbref_row.reset_index(drop=True),fbref_players.iloc[i:i+1].reset_index(drop=True)],axis=1)
            MergedSoFifaFbref_row = MergedSoFifaFbref_row.reset_index(drop=True)
            for column in fbref_players.columns.values.tolist():
                MergedSoFifaFbref_row[column] = fbref_players[column].iloc[i]
            
        # If the dataframe df_sofifa is empty, create a dataframe called MergedSoFifaFbref_row which contains the current row of fbref_players.
        else:
            MergedSoFifaFbref_row = fbref_players.iloc[i:i+1]
        # Reset the index of MergedSoFifaFbref.
        MergedSoFifaFbref.reset_index(inplace=True, drop=True)
        # Concatenate the current row of MergedSoFifaFbref_row and MergedSoFifaFbref.
        MergedSoFifaFbref = pd.concat([MergedSoFifaFbref,MergedSoFifaFbref_row])
    MergedSoFifaFbref.to_csv('./data/MergeSofifaFbref_new.csv',index=False,header=True,encoding='utf-8-sig')

MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchSquad,player_attr)

C:\Users\super\AppData\Local\Temp\ipykernel_2276\2465697214.py:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  MergedSoFifaFbref = pd.concat([MergedSoFifaFbref,MergedSoFifaFbref_row])
C:\Users\super\AppData\Local\Temp\ipykernel_2276\2465697214.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sofifa['team1_kitnum'] = df_sofifa['team1_kitnum'].astype(str)
C:\Users\super\AppData\Local\Temp\ipykernel_2276\2465697214.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [ ]:
# def MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr):
#     fbref_players = fbref_MatchPlayerStats[['player_kitnum','player_name','position','team','nationality','fbrefMatchId','age']]
#     fbref_players = fbref_players.merge(fbref_MatchInfos[['match_date','fbrefMatchId']], on='fbrefMatchId', how = 'inner')
#     fbref_players['player_name'] = fbref_players['player_name'].astype('str')
#     fbref_players['match_date'] = fbref_players['match_date'].astype('datetime64[ns]')
#     player_attr['update_date'] = player_attr['update_date'].astype('datetime64[ns]')
#     fbref_players['age'] = fbref_players['age'].astype('str')
    

# MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr)

In [ ]:
fbref_MatchInfos['match_date'] = fbref_MatchInfos['match_date'].astype('datetime64[ns]')
fbref_MatchInfos['match_date'][1]

In [ ]:

# names = ['Troy','Deeney','askjd']
# all([name in ['Troy Deeney'] for name in names])
print(fbref_MatchInfos.columns.values.tolist())
